In [94]:
import Base: +, -, *, /, exp, ^, show

# user Taylor series exp(x) ≈ Σ_{k=0}^n x^k/k!
function exp_taylor_fast(x, n)
    ret = 0
    summand = 1
    for k = 0:n
        ret += summand
        summand *= x/(k+1) # uses Float64 arithmetic
    end
    return ret
end



exp_taylor_fast (generic function with 1 method)

In [99]:
# Dual(a,b) represents a + b*ε
struct Dual{T}
    a::T
    b::T
end

function Dual(a, b)
    T = promote_type(typeof(a), typeof(b))
    Dual{T}(a, b)
end

const ε = Dual(0,1)



+(x::Real, y::Dual) = Dual(x + y.a, y.b) 
+(x::Dual, y::Real) = y + x
*(x::Real, y::Dual) = Dual(x*y.a, x*y.b)
/(y::Dual, k::Real) = Dual(y.a/k, y.b/k)
# (a+b*ε)*(c+d*ε) = a*c + (b*c + a*d)ε
*(x::Dual, y::Dual) = Dual(x.a*y.a, x.b*y.a + x.a*y.b)
+(x::Dual, y::Dual) = Dual(x.a + y.a, x.b + y.b)
-(x::Dual) = Dual(-x.a, -x.b)

function ^(x::Dual, k::Number)
    if k < 0
        error("Not implemented")
    elseif k == 1
        x
    else
        x^(k-1) * x
    end
end

exp(x::Dual) = Dual(exp(x.a), x.b*exp(x.a))
    
show(io::IO, x::Dual) = print(io, "$(x.a) + $(x.b)ε")

show (generic function with 302 methods)

In [105]:
x = 1+2im
@which x * x

*(z::Complex, w::Complex) in Base at complex.jl:290

In [ ]:
exp_taylor_fast(1 + ε, 100).b - ℯ

In [35]:
# compare with CD
h = 0.0000001; 
(exp_taylor_fast(1 + h, 100)-exp_taylor_fast(1-h, 100))/(2h) - ℯ

-2.161858692062424e-9

In [50]:

f = x -> exp(-exp(x)^2)
f(1 + ε), (f(1+h)-f(1-h))/2h


(Dual{Float64}(0.000617978989331094, -0.009132562840255837), -0.009132562839102819)

In [68]:
x = 1.1 + ε
for k = 1:1002
    x = exp((-1)^k*x)
end
x

Dual{Float64}(1.3097995858041505, -6.530413055054338e-227)

In [84]:
f = function(x)
    for k = 1:100
    x = exp((-1)^k*x)
end
x
end

#9 (generic function with 1 method)

In [91]:
cd = (setprecision(10_000) do
    h = big(2.0)^(-200)
    (f(big(1.0)+h)-f(big(1.0)))/h
end)

3.46612911524293397586185531062558111983366462004456433648697825060194612843271502522625548761222413229376715571738320700878148599406100300702591879014260725377563825923058703149448080985286466044834549505187510013649162152736859887258119283534834691738368593187062643603581302500600599715512609700855485581346719472729192439671476689718290255865328622606398214274820664810455561753355193207282157974882185118291618256846891105228520162772822965825796621728076392118055460383353238756309353385454361643617846943223776481957737396078980354607048497388847606840330437654049975529667043152693026122737055581710494827426890480366309756762374945954829585474026289646146981137447395841098554100829057652533352140647385246604269869700763552597451511551657286713269003643175218357391682141884077925667594667810147918682811281762244936421290792633749177270122472033249614025666562018103716273559330890980395119600047141521486180853946378273392711686583746926714975005046748008684389072837900278272298623498588

In [88]:
f(1 + ε)

Dual{Float64}(1.3097995858041505, 3.466129115242939e-23)

In [93]:
(f(big(1) + ε).b - cd)/cd

2.671784500968620893204614155730170747930444687512460310304847179832720182196829e-61